<h1><b>Applied Data Science</b></h1>
<p>Assignment 2</p>


<h2><b>Prepare Postal Codes</b></h2>


In [1]:
#Import Libraries
import requests
import lxml.html as lh
import pandas as pd

In [2]:
#Get the Postal Codes web page from Wikipedia
#Specify url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    #print '%d:"%s"'%(i,name)
    col.append((name,[]))

In [4]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=data ##int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
[len(C) for (title,C) in col]

[181, 181, 181]

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [8]:
df.head()


,Postal code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n


In [9]:
## Strip the '\n' suffixes
df.columns = [x.strip().replace('\n', '') for x in df.columns]
df = df.apply(lambda S:S.str.strip('\n'))
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [10]:
# Drop rows where Borough contains("Not assigned")]
df = df[~df['Borough'].str.contains('Not assigned')]
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [11]:
#Combine the Neighborhoods for duplicate postal codes
df.groupby(['Postal code'])['Neighborhood'].apply(','.join).reset_index()
df.head(20)

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [12]:
df.shape

(104, 3)

<h2><b>Add in the Latitude and Longitude data</b></h2>

In [13]:
filename='https://cocl.us/Geospatial_data'
df_latlong = pd.read_csv(filename)

In [14]:
df_latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
#Combine the lat long data and the postal code data frame
df_comb = pd.merge(df, df_latlong, left_on='Postal code',right_on='Postal Code', how='left', indicator='True')
df_comb.head(20)

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude,True
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656,both
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572,both
2,M5A,Downtown Toronto,Regent Park / Harbourfront,M5A,43.654260,-79.360636,both
3,M6A,North York,Lawrence Manor / Lawrence Heights,M6A,43.718518,-79.464763,both
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,M7A,43.662301,-79.389494,both
5,M9A,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242,both
6,M1B,Scarborough,Malvern / Rouge,M1B,43.806686,-79.194353,both
7,M3B,North York,Don Mills,M3B,43.745906,-79.352188,both
8,M4B,East York,Parkview Hill / Woodbine Gardens,M4B,43.706397,-79.309937,both
9,M5B,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937,both


In [16]:
#Drop the 'Postal Code' column
df_comb.drop(['Postal Code','True'], axis = 1, inplace=True)
df_comb.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [17]:
df_comb.shape

(104, 5)

<h2><b>Final analysis and observations</b></h2>

In [18]:
#Import Libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# uncomment this line if you haven't completed the Foursquare API lab
#!pip install geopy  
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
# uncomment this line if you haven't completed the Foursquare API lab
#!pip install folium 
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


In [21]:
#Define Foursquare credentials
CLIENT_ID = 'WU2ZUP2ZCZA4P1T22UTHY42HN2JJE5RFKDTJSXBFR1HB22PW' # your Foursquare ID
CLIENT_SECRET = 'EHBZ5ZG2P3TLSJUB3PIWFQ0UHV1LFNN4DV5U4QN0QCPZ5C3D' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WU2ZUP2ZCZA4P1T22UTHY42HN2JJE5RFKDTJSXBFR1HB22PW
CLIENT_SECRET:EHBZ5ZG2P3TLSJUB3PIWFQ0UHV1LFNN4DV5U4QN0QCPZ5C3D


In [22]:
#Lets filter out Central Toronto

c_toronto_data = df_comb[df_comb['Borough'] == 'Central Toronto'].reset_index(drop=True)
c_toronto_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


In [23]:
# Let get the first Neigborhoods data
c_toronto_data.loc[0, 'Neighborhood']

'Lawrence Park'

In [24]:
neighborhood_latitude = c_toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = c_toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = c_toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


In [25]:
#Let look at the 1st 100 hundred venues for the above
LIMIT = 100 

radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WU2ZUP2ZCZA4P1T22UTHY42HN2JJE5RFKDTJSXBFR1HB22PW&client_secret=EHBZ5ZG2P3TLSJUB3PIWFQ0UHV1LFNN4DV5U4QN0QCPZ5C3D&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e96d0b0618f43001b4907e1'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledL

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


<h3><i> Lets get the neigborhoods data for Central Toronto</i></h3>

In [30]:
#First define the function
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
#Create the dataframe
c_toronto_venues = getNearbyVenues(names=c_toronto_data['Neighborhood'],
                                   latitudes=c_toronto_data['Latitude'],
                                   longitudes=c_toronto_data['Longitude']
                                  )


Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
North Toronto West
The Annex / North Midtown / Yorkville
Davisville
Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park


In [32]:
print(c_toronto_venues.shape)
c_toronto_venues.head()

(116, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden
4,Roselawn,43.711695,-79.416936,Havergal College,43.712108,-79.411680,Music Venue


In [33]:
c_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,34,34,34,34,34,34
Davisville North,11,11,11,11,11,11
Forest Hill North & West,4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
Moore Park / Summerhill East,3,3,3,3,3,3
North Toronto West,21,21,21,21,21,21
Roselawn,2,2,2,2,2,2
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park,16,16,16,16,16,16
The Annex / North Midtown / Yorkville,22,22,22,22,22,22


In [34]:
# Analyse each neighborhood
# one hot encoding
c_toronto_onehot = pd.get_dummies(c_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
c_toronto_onehot['Neighborhood'] = c_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [c_toronto_onehot.columns[-1]] + list(c_toronto_onehot.columns[:-1])
c_toronto_onehot = c_toronto_onehot[fixed_columns]

c_toronto_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dance Studio,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Music Venue,New American Restaurant,Park,Pharmacy,Pizza Place,Playground,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
c_toronto_onehot.shape

(116, 67)

In [36]:
c_toronto_grouped = c_toronto_onehot.groupby('Neighborhood').mean().reset_index()
c_toronto_grouped

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dance Studio,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Music Venue,New American Restaurant,Park,Pharmacy,Pizza Place,Playground,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.029412,0.000000,0.0000,0.0000,0.000000,0.029412,0.000000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.088235,0.029412,0.000000,0.029412,0.000000,0.000000,0.0000,0.0,0.029412,0.029412,0.029412,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.029412,0.058824,0.029412,0.00,0.0000,0.000000,0.000000,0.000000,0.0,0.029412,0.029412,0.029412,0.058824,0.000000,0.000000,0.029412,0.000000,0.088235,0.029412,0.000000,0.000000,0.0000,0.000000,0.0000,0.058824,0.000000,0.029412,0.029412,0.00,0.000000,0.0000,0.000000
1,Davisville North,0.000000,0.090909,0.000000,0.0000,0.0000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.090909,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.090909,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000
2,Forest Hill North & West,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.250000,0.000000,0.000000,0.000000,0.25,0.000000,0.0000,0.000000
3,Lawrence Park,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.333333,0.000000,0.000000,0.00,0.000000,0.0000,0.000000
4,Moore Park / Summerhill East,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.333333,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000
5,North Toronto West,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.142857,0.0952

In [37]:
#Print top 5 venues per neighborhood

num_top_venues = 5

for hood in c_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = c_toronto_grouped[c_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0      Sandwich Place  0.09
1        Dessert Shop  0.09
2                 Gym  0.06
3  Italian Restaurant  0.06
4                Café  0.06


----Davisville North----
               venue  freq
0              Hotel  0.09
1                Gym  0.09
2  Food & Drink Shop  0.09
3   Department Store  0.09
4       Dance Studio  0.09


----Forest Hill North & West----
                       venue  freq
0           Sushi Restaurant  0.25
1                      Trail  0.25
2              Jewelry Store  0.25
3                       Park  0.25
4  Middle Eastern Restaurant  0.00


----Lawrence Park----
                       venue  freq
0                   Bus Line  0.33
1                       Park  0.33
2                Swim School  0.33
3  Middle Eastern Restaurant  0.00
4                 Playground  0.00


----Moore Park / Summerhill East----
                 venue  freq
0           Playground  0.33
1                 Park  0.33
2          Summer C

In [38]:
#Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
#Disply top 10 venues per neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = c_toronto_grouped['Neighborhood']

for ind in np.arange(c_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(c_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Sandwich Place,Café,Gym,Coffee Shop,Sushi Restaurant,Pizza Place,Italian Restaurant,Pharmacy,New American Restaurant
1,Davisville North,Dance Studio,Sandwich Place,Hotel,Food & Drink Shop,Park,Pizza Place,Department Store,Convenience Store,Gym,Asian Restaurant
2,Forest Hill North & West,Jewelry Store,Park,Trail,Sushi Restaurant,Fried Chicken Joint,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop
3,Lawrence Park,Bus Line,Park,Swim School,Yoga Studio,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint
4,Moore Park / Summerhill East,Summer Camp,Park,Playground,Yoga Studio,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop


In [40]:
# Cluster info using KMeans
# set number of clusters
kclusters = 5

c_toronto_grouped_clustering = c_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(c_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 2, 3, 0, 1, 0, 0])

In [41]:
#Create dataframe of the data according to clusters

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

c_toronto_merged = c_toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
c_toronto_merged = c_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

c_toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Bus Line,Park,Swim School,Yoga Studio,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Garden,Music Venue,Yoga Studio,Department Store,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Gas Station,Fried Chicken Joint
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Dance Studio,Sandwich Place,Hotel,Food & Drink Shop,Park,Pizza Place,Department Store,Convenience Store,Gym,Asian Restaurant
3,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,4,Jewelry Store,Park,Trail,Sushi Restaurant,Fried Chicken Joint,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Chinese Restaurant,Gym / Fitness Center,Health & Beauty Service,Fast Food Restaurant,Diner,Mexican Restaurant,Dessert Shop


<h3><i>Examine the clusters</i></h3>

In [42]:
#Cluster 1
c_toronto_merged.loc[c_toronto_merged['Cluster Labels'] == 0, c_toronto_merged.columns[[1] + list(range(5, c_toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Central Toronto,0,Dance Studio,Sandwich Place,Hotel,Food & Drink Shop,Park,Pizza Place,Department Store,Convenience Store,Gym,Asian Restaurant
4,Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Chinese Restaurant,Gym / Fitness Center,Health & Beauty Service,Fast Food Restaurant,Diner,Mexican Restaurant,Dessert Shop
5,Central Toronto,0,Café,Sandwich Place,Coffee Shop,History Museum,Pub,BBQ Joint,Burger Joint,Cosmetics Shop,Donut Shop,Indian Restaurant
6,Central Toronto,0,Dessert Shop,Sandwich Place,Café,Gym,Coffee Shop,Sushi Restaurant,Pizza Place,Italian Restaurant,Pharmacy,New American Restaurant
8,Central Toronto,0,Coffee Shop,Pub,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Liquor Store,Pizza Place,Restaurant,American Restaurant


In [43]:
#Cluster 2
c_toronto_merged.loc[c_toronto_merged['Cluster Labels'] == 1, c_toronto_merged.columns[[1] + list(range(5, c_toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,1,Garden,Music Venue,Yoga Studio,Department Store,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Gas Station,Fried Chicken Joint


In [44]:
#Cluster 3
c_toronto_merged.loc[c_toronto_merged['Cluster Labels'] == 2, c_toronto_merged.columns[[1] + list(range(5, c_toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,2,Bus Line,Park,Swim School,Yoga Studio,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint


In [45]:
#Cluster 4
c_toronto_merged.loc[c_toronto_merged['Cluster Labels'] == 3, c_toronto_merged.columns[[1] + list(range(5, c_toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Central Toronto,3,Summer Camp,Park,Playground,Yoga Studio,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop


In [46]:
#Cluster 5
c_toronto_merged.loc[c_toronto_merged['Cluster Labels'] == 4, c_toronto_merged.columns[[1] + list(range(5, c_toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Central Toronto,4,Jewelry Store,Park,Trail,Sushi Restaurant,Fried Chicken Joint,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop


<h3><i>Visualise the results</i></h3>

In [48]:
# create map

latitude=43.711695 
longitude=-79.416936

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(c_toronto_merged['Latitude'], c_toronto_merged['Longitude'], c_toronto_merged['Neighborhood'], c_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters